# 動作確認用のファイル

In [1]:
import pandas as pd
import math
import pickle

In [2]:
tanka = pd.read_csv("kindai.csv",header=None)
waka = pd.read_csv("waka_half.csv",header=None)

In [3]:
print(tanka.info())
print(tanka.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13963 entries, 0 to 13962
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       13963 non-null  object
dtypes: object(1)
memory usage: 109.2+ KB
None
                                     0
0  あああらき なげきやながき くろかみも ひとときにして あせやしぬべし
1  あああらき なげきやなんじ れはかりは てんもうごけと なくにもにたる
2  ああきみは いたましきまで ようちなる なさけにゑひぬ かなしからずや
3  ああきみは かたるにたらず よわきじん よわきこいをば もてあそぶかな
4  ああきみは すむむねもなき さびしさを かたるかなみだ かわきしめもて


In [4]:
print(waka.info())
print(waka.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14292 entries, 0 to 14291
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       14292 non-null  object
dtypes: object(1)
memory usage: 111.8+ KB
None
                                     0
0  あかからは みるへきものを かりかねの いつこはかりに なきてゆくらむ
1  あかさりし おもかけはかり うつしても ちきりはさてや やまのゐのみつ
2  あかさりし きみかにほひの こひしさに うめのはなをそ けさはをりつる
3  あかさりし そてかとにほふ うめかかに おもひなくさむ あかつきのそら
4  あかさりし そてのわかれの なみたより かたみかほなる つきそみにそふ


In [17]:
def vectorize_df(df):
    list_df = df.apply(split_text,axis=1)
    tri_gram_df = list_df.apply(tri_gram)
    vector = calc_vector(tri_gram_df)
    return vector

def split_text(text_record):
    return text_record.loc[0].split(" ")

def tri_gram(text_list):
    ans = []
    for text in text_list:
        gram = []
        for i in range(len(text)-2):
            gram.append(text[i:i+3])
        ans.extend(gram)
    return ans

def calc_vector(gram_df):
    vector = {}
    vector["vector"] = {}
    vector["id"] = {}
    r2 = 0
    id = 0
    for line in gram_df:
        for word in line:
            if word in vector["vector"].keys():
                vector["vector"][word] += 1
            else:
                vector["vector"][word] = 1
                vector["id"][word] = id
            id += 1

    for key in vector["vector"].keys():
        r2 += vector["vector"][key]**2

    r = math.sqrt(r2)
    for key in vector["vector"].keys():
        vector["vector"][key] /= r

    return vector

In [18]:
waka_vector = vectorize_df(waka)
tanka_vector = vectorize_df(tanka)

In [19]:
waka_path = "waka_vector.pkl"
tanka_path = "tanka_vector.pkl"

with open(waka_path,"wb") as f1:
    pickle.dump(waka_vector, f1)

with open(tanka_path,"wb") as f2:
    pickle.dump(tanka_vector, f2)

In [20]:
with open(tanka_path,"rb") as f:
    v = pickle.load(f)

In [22]:
v

{'vector': {'あああ': 0.00115254140161483,
  'ああら': 0.0008644060512111224,
  'あらき': 0.005186436307266735,
  'なげき': 0.015271173571396497,
  'げきや': 0.0008644060512111224,
  'きやな': 0.0014406767520185374,
  'やなが': 0.0014406767520185374,
  'ながき': 0.018728797776240987,
  'くろか': 0.00576270700807415,
  'ろかみ': 0.0060508423584778575,
  'かみも': 0.0025932181536333673,
  'ひとと': 0.027949128989159626,
  'ととき': 0.012101684716955715,
  'ときに': 0.02506777548512255,
  'きにし': 0.013254226118570545,
  'にして': 0.08240671021546034,
  'あせや': 0.0008644060512111224,
  'せやし': 0.0002881353504037075,
  'やしぬ': 0.000576270700807415,
  'しぬべ': 0.0037457595552481972,
  'ぬべし': 0.0031694888544407823,
  'やなん': 0.0002881353504037075,
  'なんじ': 0.0020169474528259524,
  'れはか': 0.002881353504037075,
  'はかり': 0.0037457595552481972,
  'かりは': 0.0115254140161483,
  'てんも': 0.00115254140161483,
  'んもう': 0.000576270700807415,
  'もうご': 0.0020169474528259524,
  'うごけ': 0.0054745716576704425,
  'ごけと': 0.0002881353504037075,
  'なくに': 0.025067775